In [2]:
import pandas as pd
import xml.etree.ElementTree as ET
from sqlalchemy import text
from connection import connect
from utils.to_spanish import to_spanish

In [3]:
# Conexión
inspector_co, etl_conn, co_sa = connect()

Esquemas encontrados: ['hr', 'person', 'production', 'public', 'purchasing', 'sales']

--- Tablas de Negocio Encontradas ---
Esquema 'hr' (6 tablas):
  > ['department', 'employee', 'employee_department_history', 'employee_pay_history', 'job_candidate']...
Esquema 'person' (13 tablas):
  > ['business_entity', 'address', 'address_type', 'business_entity_address', 'business_entity_contact']...
Esquema 'production' (25 tablas):
  > ['illustration', 'bill_of_materials', 'culture', 'document', 'location']...
Esquema 'public' (4 tablas):
  > ['awbuild_version', 'database_log', 'error_log', 'sysdiagrams']...
Esquema 'purchasing' (5 tablas):
  > ['product_vendor', 'purchase_order_detail', 'purchase_order_header', 'ship_method', 'vendor']...
Esquema 'sales' (19 tablas):
  > ['country_region_currency', 'credit_card', 'customer', 'currency', 'currency_rate']...
Tables de datos en la dw (debe estar vacia al iniciar la conexión
[]


# Data exploration

In [6]:
inspector_co.get_columns('person', schema='person')

[{'name': 'business_entity_id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'person_type',
  'type': CHAR(length=2),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'name_style',
  'type': BOOLEAN(),
  'nullable': False,
  'default': 'false',
  'autoincrement': False,
  'comment': None},
 {'name': 'title',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'first_name',
  'type': TEXT(),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'middle_name',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'last_name',
  'type': TEXT(),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'suffix',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoi

In [8]:
# Lista de tablas que queremos inspeccionar
candidate_tables = [
    ('person', 'person'),
    ('sales', 'customer'),
    ('person', 'address'),
    ('person', 'state_province'),
    ('person', 'email_address'),
    ('person', 'person_phone'),
    ('sales', 'sales_territory'),
    ('sales', 'sales_order_header')
]

for schema, table in candidate_tables:
    try:
        cols = inspector_co.get_columns(table, schema=schema)
        print(f"\n{schema}.{table} -> {len(cols)} columnas")
        for c in cols:
            print(f"  - {c['name']:30} {c['type']}")
    except Exception as e:
        print(f"\nNo se pudo obtener columnas de {schema}.{table}: {e}")



person.person -> 13 columnas
  - business_entity_id             INTEGER
  - person_type                    CHAR(2)
  - name_style                     BOOLEAN
  - title                          TEXT
  - first_name                     TEXT
  - middle_name                    TEXT
  - last_name                      TEXT
  - suffix                         TEXT
  - email_promotion                INTEGER
  - additional_contact_info        TEXT
  - demographics                   TEXT
  - rowguid                        CHAR(36)
  - modified_date                  TIMESTAMP

sales.customer -> 7 columnas
  - customer_id                    INTEGER
  - person_id                      INTEGER
  - store_id                       INTEGER
  - territory_id                   INTEGER
  - account_number                 TEXT
  - rowguid                        CHAR(36)
  - modified_date                  TIMESTAMP

person.address -> 9 columnas
  - address_id                     INTEGER
  - address_line_1       

Consulta de union de valores principales

    SELECT
        c.account_number as alternate_key,
        p.title,
        p.first_name,
        p.middle_name,
        p.last_name,
        p.suffix,
        p.name_style
        p.demographics,
        e.email_address,
        pp.phone_number,
        a.address_line_1,
        a.address_line_2
    FROM sales.customer c
    INNER JOIN person.person p
        ON c.person_id = p.business_entity_id
    INNER JOIN person.email_address e
        ON p.business_entity_id = e.business_entity_id
    INNER JOIN person.person_phone pp
        ON p.business_entity_id = pp.business_entity_id
    INNER JOIN person.business_entity_address bea
        ON p.business_entity_id = bea.business_entity_id
    INNER JOIN person.address a
        ON bea.address_id = a.address_id


Tabla relacionada al cliente

    CREATE TABLE IF NOT EXISTS dw.dim_customer (
        customer_key INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
        customer_alternate_key TEXT,
        geography_key INT,
        title TEXT,
        first_name TEXT,
        middle_name TEXT,
        last_name TEXT,
        suffix TEXT,
        name_style BOOLEAN,
        birth_date DATE,
        gender VARCHAR(1),
        marital_status VARCHAR(1),
        yearly_income TEXT,
        total_children INT,
        number_children_at_home INT,
        education TEXT,
        spanish_education TEXT,
        occupation TEXT,
        spanish_occupation TEXT,
        home_owner_flag BOOLEAN,
        number_cars_owned INT,
        email_address TEXT,
        phone_number TEXT,
        address_line_1 TEXT,
        address_line_2 TEXT,
        date_first_purchase DATE,
        commute_distance TEXT
     );

# Extract

In [56]:
# Recuperando dimension de geografía

df_geo_with_keys = pd.read_sql(
    text("""
        SELECT geography_key, city, postal_code, state_province_code, country_region_code
    FROM dw.dim_geography;
    """),
    etl_conn
)

print(f"Se recuperaron {len(df_geo_with_keys)} registros de geografía.")
print(df_geo_with_keys.head())


Se recuperaron 655 registros de geografía.
   geography_key          city postal_code state_province_code  \
0              1   Lake George       12845                 NY    
1              2    North Ryde        2113                 NSW   
2              3          Clay       13041                 NY    
3              4  Bell Gardens       90201                 CA    
4              5       Burbank       91502                 CA    

  country_region_code  
0                  US  
1                  AU  
2                  US  
3                  US  
4                  US  


In [58]:
query_dimension = text("""

--- SOLO COMPRAS HECHAS DESDE INTERNET
WITH internet_customers AS (
    SELECT DISTINCT customer_id
    FROM sales.sales_order_header
    WHERE online_order_flag = true
)


SELECT
    c.account_number AS customer_alternate_key,
    p.title,
    p.first_name,
    p.middle_name,
    p.last_name,
    p.suffix,
    p.name_style,
    p.demographics,
    e.email_address,
    rp.phone_number,
    a.address_line_1,
    a.address_line_2,

    a.city,
    a.postal_code,
    sp.state_province_code,
    cr.country_region_code
FROM
    sales.customer AS c
    INNER JOIN internet_customers AS ic
        ON c.customer_id = ic.customer_id
    INNER JOIN person.person AS p
        ON c.person_id = p.business_entity_id
    INNER JOIN person.email_address AS e
        ON p.business_entity_id = e.business_entity_id
    LEFT JOIN person.person_phone AS rp
        ON p.business_entity_id = rp.business_entity_id
    INNER JOIN person.business_entity_address AS bea
        ON p.business_entity_id = bea.business_entity_id
    INNER JOIN person.address AS a
        ON bea.address_id = a.address_id
    INNER JOIN person.state_province AS sp
        ON a.state_province_id = sp.state_province_id
    INNER JOIN person.country_region AS cr
        ON sp.country_region_code = cr.country_region_code
    INNER JOIN person.address_type AS at
        ON bea.address_type_id = at.address_type_id
WHERE
    at.name = 'Home';
""")

df_ctm = pd.read_sql(query_dimension, co_sa)


In [59]:
print(df_ctm.columns)
print(df_ctm.shape)

Index(['customer_alternate_key', 'title', 'first_name', 'middle_name',
       'last_name', 'suffix', 'name_style', 'demographics', 'email_address',
       'phone_number', 'address_line_1', 'address_line_2', 'city',
       'postal_code', 'state_province_code', 'country_region_code'],
      dtype='object')
(18484, 16)


# Transform

In [60]:
print(df_ctm['demographics'].apply(type).value_counts())

demographics
<class 'str'>    18484
Name: count, dtype: int64


In [61]:
# Extraer elementos del xml demographics

def extract_text_or_none(element):
    return element.text if element is not None else None

none_xml_columns_dict = {
            'birth_date': None,
            'gender' : None,
            'marital_status' : None,
            'yearly_income' : None,
            'education' : None,
            'occupation' : None,
            'home_owner_flag' : None,
            'number_cars_owned' : None,
            'date_first_purchase' : None,
            'commute_distance' : None,
        }

def extract_demographics(xml_string):
    if pd.isna(xml_string):
        return none_xml_columns_dict

    try:
        root = ET.fromstring(xml_string)
        ns = {'ns': root.tag.split('}')[0].strip('{')}

        def namespace_find(tag):
            el = root.find(f'ns:{tag}', ns)
            return el.text if el is not None else None

        return {
            'birth_date': namespace_find('BirthDate'),
            'gender': namespace_find('Gender'),
            'marital_status': namespace_find('MaritalStatus'),
            'yearly_income': namespace_find('YearlyIncome'),
            'total_children': namespace_find('TotalChildren'),
            'number_children_at_home': namespace_find('NumberChildrenAtHome'),
            'education': namespace_find('Education'),
            'occupation': namespace_find('Occupation'),
            'home_owner_flag': namespace_find('HomeOwnerFlag'),
            'number_cars_owned': namespace_find('NumberCarsOwned'),
            'date_first_purchase': namespace_find('DateFirstPurchase'),
            'commute_distance': namespace_find('CommuteDistance'),
        }

    except Exception:
        return none_xml_columns_dict

demographics_df = df_ctm['demographics'].apply(extract_demographics).apply(pd.Series)
df_ctm = pd.concat([df_ctm.drop('demographics', axis=1), demographics_df], axis=1)


In [62]:
# Pasando valores a booleanos
df_ctm['home_owner_flag'] = df_ctm['home_owner_flag'].astype(bool)

df_ctm.tail()

,customer_alternate_key,title,first_name,middle_name,last_name,suffix,name_style,email_address,phone_number,address_line_1,...,marital_status,yearly_income,total_children,number_children_at_home,education,occupation,home_owner_flag,number_cars_owned,date_first_purchase,commute_distance
18479,AW00019379,None,Crystal,None,Guo,None,False,crystal18@adventure-works.com,1 (11) 500 555-0171,988 Mt. Everest Court,...,S,0-25000,0,0,Partial High School,Manual,True,2,2004-04-19Z,0-1 Miles
18480,AW00013933,None,Isabella,F,Richardson,None,False,isabella91@adventure-works.com,910-555-0166,7413 Alpine Drive,...,M,50001-75000,1,0,Partial College,Skilled Manual,True,1,2003-08-30Z,2-5 Miles
18481,AW00024634,None,Crystal,S,He,None,False,crystal19@adventure-works.com,813-555-0148,4764 East Avenue,...,M,50001-75000,3,0,Bachelors,Management,True,2,2004-04-12Z,10+ Miles
18482,AW00021127,None,Crystal,None,Zheng,None,False,crystal20@adventure-works.com,1 (11) 500 555-0171,"34334, rue Jean Mermoz",...,S,25001-50000,0,0,Partial College,Clerical,True,1,2004-02-15Z,2-5 Miles
18483,AW00027980,None,Crystal,None,Hu,None,False,crystal21@adventure-works.com,1 (11) 500 555-0126,6022 La Salle Ct.,...,M,75001-100000,0,0,Bachelors,Professional,True,4,2003-11-17Z,10+ Miles


In [63]:
# Verificando valores nulos
columns_with_nulls = df_ctm.columns[df_ctm.isnull().any()].tolist()
print("Columnas con valores nulos")
print(columns_with_nulls)

Columnas con valores nulos
['title', 'middle_name', 'suffix', 'address_line_2']


In [64]:
spanish_education_dict = to_spanish(df_ctm, 'education')
spanish_occupations_dict = to_spanish(df_ctm, 'occupation')

print(spanish_education_dict)
print(spanish_occupations_dict)

{'Graduate Degree': 'Licenciatura', 'Bachelors ': 'Licenciaturas', 'Partial College': 'Universidad Parcial', 'Partial High School': 'Preparatoria Parcial', 'High School': 'Escuela secundaria'}
{'Clerical': 'Clerical', 'Professional': 'Profesional', 'Skilled Manual': 'Manual especializado', 'Manual': 'Manual', 'Management': 'Gestión'}


In [65]:
df_ctm['spanish_education'] = df_ctm['education'].map(spanish_education_dict)
df_ctm['spanish_occupation'] = df_ctm['occupation'].map(spanish_occupations_dict)

In [66]:
df_ctm.head()

,customer_alternate_key,title,first_name,middle_name,last_name,suffix,name_style,email_address,phone_number,address_line_1,...,total_children,number_children_at_home,education,occupation,home_owner_flag,number_cars_owned,date_first_purchase,commute_distance,spanish_education,spanish_occupation
0,AW00011377,Mr.,David,R.,Robinett,None,False,david22@adventure-works.com,238-555-0100,Pappelallee 6667,...,4,0,Graduate Degree,Clerical,True,0,2003-09-01Z,0-1 Miles,Licenciatura,Clerical
1,AW00011913,Ms.,Rebecca,A.,Robinson,None,False,rebecca3@adventure-works.com,648-555-0100,1861 Chinquapin Ct,...,3,3,Bachelors,Professional,True,1,2004-06-05Z,5-10 Miles,Licenciaturas,Profesional
2,AW00011952,Ms.,Dorothy,B.,Robinson,None,False,dorothy3@adventure-works.com,423-555-0100,4693 Mills Dr.,...,2,0,Partial College,Skilled Manual,True,2,2002-04-07Z,1-2 Miles,Universidad Parcial,Manual especializado
3,AW00020164,Ms.,Carol Ann,F.,Rockne,None,False,carolann0@adventure-works.com,439-555-0100,1312 Skycrest Drive,...,1,0,Bachelors,Clerical,True,0,2001-10-27Z,0-1 Miles,Licenciaturas,Clerical
4,AW00020211,Mr.,Scott,M.,Rodgers,None,False,scott10@adventure-works.com,989-555-0100,9860 Brookview Drive,...,2,2,Bachelors,Professional,True,1,2002-04-18Z,2-5 Miles,Licenciaturas,Profesional


In [67]:
# Vinculando dim_customer con dim_geography

linking_columns = ['city', 'postal_code', 'state_province_code', 'country_region_code']

df_customer_final = pd.merge(
    df_ctm,
    df_geo_with_keys,
    on=linking_columns,
    how='left'
)

print("Columnas después del merge:", df_customer_final.columns)
print(df_customer_final[['first_name', 'city', 'geography_key']].head())

Columnas después del merge: Index(['customer_alternate_key', 'title', 'first_name', 'middle_name',
       'last_name', 'suffix', 'name_style', 'email_address', 'phone_number',
       'address_line_1', 'address_line_2', 'city', 'postal_code',
       'state_province_code', 'country_region_code', 'birth_date', 'gender',
       'marital_status', 'yearly_income', 'total_children',
       'number_children_at_home', 'education', 'occupation', 'home_owner_flag',
       'number_cars_owned', 'date_first_purchase', 'commute_distance',
       'spanish_education', 'spanish_occupation', 'geography_key'],
      dtype='object')
  first_name           city  geography_key
0      David       Solingen            250
1    Rebecca        Seaford            622
2    Dorothy        Geelong            171
3  Carol Ann      Lancaster            300
4      Scott  East Brisbane            178


# Load

In [68]:
final_columns = [
    'customer_alternate_key', 'geography_key', 'title', 'first_name', 'middle_name',
    'last_name', 'suffix', 'name_style', 'birth_date', 'gender', 'marital_status',
    'yearly_income', 'total_children', 'number_children_at_home', 'education',
    'spanish_education', 'occupation', 'spanish_occupation', 'home_owner_flag',
    'number_cars_owned', 'email_address', 'phone_number', 'address_line_1', 'address_line_2',
    'date_first_purchase', 'commute_distance'
]

df_to_load = df_customer_final[final_columns]

df_to_load.head()

,customer_alternate_key,geography_key,title,first_name,middle_name,last_name,suffix,name_style,birth_date,gender,...,occupation,spanish_occupation,home_owner_flag,number_cars_owned,email_address,phone_number,address_line_1,address_line_2,date_first_purchase,commute_distance
0,AW00011377,250,Mr.,David,R.,Robinett,None,False,1961-02-23Z,M,...,Clerical,Clerical,True,0,david22@adventure-works.com,238-555-0100,Pappelallee 6667,None,2003-09-01Z,0-1 Miles
1,AW00011913,622,Ms.,Rebecca,A.,Robinson,None,False,1965-06-11Z,F,...,Professional,Profesional,True,1,rebecca3@adventure-works.com,648-555-0100,1861 Chinquapin Ct,None,2004-06-05Z,5-10 Miles
2,AW00011952,171,Ms.,Dorothy,B.,Robinson,None,False,1954-09-23Z,M,...,Skilled Manual,Manual especializado,True,2,dorothy3@adventure-works.com,423-555-0100,4693 Mills Dr.,None,2002-04-07Z,1-2 Miles
3,AW00020164,300,Ms.,Carol Ann,F.,Rockne,None,False,1943-07-15Z,M,...,Clerical,Clerical,True,0,carolann0@adventure-works.com,439-555-0100,1312 Skycrest Drive,None,2001-10-27Z,0-1 Miles
4,AW00020211,178,Mr.,Scott,M.,Rodgers,None,False,1968-05-15Z,M,...,Professional,Profesional,True,1,scott10@adventure-works.com,989-555-0100,9860 Brookview Drive,None,2002-04-18Z,2-5 Miles


In [69]:
df_to_load.shape

(18484, 26)

In [70]:
df_to_load.to_sql(
    'dim_customer',
    etl_conn,
    schema='dw',
    if_exists='append',
    index=False
)

484